In [ ]:
from multiprocessing import Pool
from time import sleep

# Prep
First of all, consider rewriting your for loops...

In [ ]:
for i in range(10):
    sleep(0.5)
    print(f'I can count to {i}')

... in a more functional way, using a function...

In [ ]:
def learn_to_count(i: int) -> str:
    sleep(0.5)
    return f'I can count to {i}'

... and a call to `map`, which maps your inputs (`range(10)`) over the function above (`learn_to_count`).

Note that `map` returns an iterator, i.e. is not evaluated right away. Therefore I cast it to a `list` in order to execute:

In [ ]:
%time list(map(learn_to_count, range(10)))

# Parallelisation
Having adapted the functional style, it's really easy to parallelise the executing, by just using `multiprocessing.Pool.map`:

1. create a `Pool` (optionally with number of processes to use)
1. use `Pool.map` instead of your usual `map`
1. It runs in parallel, awesome! -- see how much faster this runs now!

In [ ]:
p = Pool(processes=5)
%time p.map(learn_to_count, range(10))

depending on your application, you might want to increase the amount of processes used to make it even faster...

In [ ]:
p = Pool(processes=10)
%time p.map(learn_to_count, range(10))

# Asynchronous parallelisation
Going one step further, you can run your job asynchronously, by using `Pool.map_async` and `.get` on the resulting `MapResult`:

In [ ]:
p = Pool(5)
r = p.map_async(learn_to_count, range(10))
%time r.get()

Between `map_async` and `get`, you can run other code, which will run in parallel to the asynchronous run:

In [ ]:
r = p.map_async(learn_to_count, range(10))

sleep(0.5)

%time r.get()

As you can see, only 0.5 seconds are spent on the `get`, as the `map_async` has already been running in parallel with our `sleep(0.5)` for 0.5 seconds. 

# Note the order
Note that the parallelised jobs do not run in order, i.e. any side effects (like printing or shared state) are generally a bad idea:

In [ ]:
def learn_to_count_and_print(i: int) -> str:
    sleep(0.5)
    print(f'I can count to {i}')

In [ ]:
p = Pool(processes=10)
%time p.map(learn_to_count_and_print, range(10))